In [ ]:
import pandas as pd
import sys

# Add your system path here
sys_path = r'C:\Users\Max Tost\Desktop\Notebooks\PowerPrediction\ml-project-2-powerpredictors'
sys.path.append(sys_path)

from helpers import *

### Loading the data in a dataframe

In [ ]:
path = sys_path + r'\epfl_campus_demand.csv'
data = LoadData(path)

# Showing the relative amount of data that are Nan in a column
relnan(data, 'estimated_demand')

0.38233132441569895

### Checking which sections are Nans

In [ ]:
nan_sections = return_nan_sections(data, 'estimated_demand') # Returns array with the first and last index of the data frame where the values are nan

# Inspecting the outcome
nan_n = 10
data['estimated_demand'][nan_sections[nan_n][0]-1:nan_sections[nan_n][1]+1]

5480    6274.005092
5481            NaN
5482            NaN
5483            NaN
5484            NaN
5485            NaN
5486            NaN
5487            NaN
5488            NaN
5489            NaN
5490            NaN
5491            NaN
5492            NaN
5493            NaN
5494            NaN
5495            NaN
5496            NaN
5497            NaN
5498            NaN
5499            NaN
5500            NaN
5501            NaN
5502            NaN
5503            NaN
5504            NaN
5505            NaN
5506            NaN
5507            NaN
5508            NaN
5509            NaN
5510            NaN
5511            NaN
5512            NaN
5513            NaN
5514            NaN
5515            NaN
5516            NaN
5517            NaN
5518            NaN
5519            NaN
5520            NaN
5521            NaN
5522            NaN
5523            NaN
5524            NaN
5525            NaN
5526            NaN
5527            NaN
5528            NaN
5529            NaN


Here its included one element before and behind the returned indices to check that it worked, which it did. Now I am happy.

## Creating features and targets to train the network
Here we will cut the whole data in slices of 7 days, which will be the features. \
The value of the power for the first hour of the 8th day should be the target. \
Then we will save them as features and targets to use them with pytorch